# 🤖 VERL + LangGraph Multi-Agent Coding Framework - Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/multiminddev/coding-framework/blob/main/colab_setup_and_demo.ipynb)

This notebook sets up and demonstrates the VERL + LangGraph Multi-Agent Coding Framework in Google Colab with HuggingFace models for local inference.

## Features Demonstrated:
- 🔧 **Code Generator Agent**: Creates code solutions from problem descriptions
- 📝 **Code Reviewer Agent**: Reviews code for quality, security, and performance
- ⚡ **Code Executor Agent**: Tests and validates code execution
- 🔄 **LangGraph Orchestration**: Multi-agent workflow coordination
- 🧠 **HuggingFace Models**: Local model inference without API keys

## 🚀 Setup and Installation

Let's start by setting up the environment and installing dependencies.

In [ ]:
# Mount Google Drive to save models and results
from google.colab import drive
drive.mount('/content/drive')

# Create directories for our project
import os
os.makedirs('/content/drive/MyDrive/coding_framework', exist_ok=True)
os.makedirs('/content/model_cache', exist_ok=True)
os.makedirs('/content/data', exist_ok=True)

print("✅ Google Drive mounted and directories created")

In [ ]:
# Check system resources
!nvidia-smi
!free -h
!df -h

import torch
print(f"\n🔥 CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Clone the repository
!git clone https://github.com/multiminddev/coding-framework.git /content/coding-framework
%cd /content/coding-framework

print("✅ Repository cloned successfully")

In [ ]:
# Install Colab-optimized dependencies
!pip install -r requirements-colab.txt

# Install the framework in development mode
!pip install -e .

print("✅ Dependencies installed successfully")

In [ ]:
# Set up environment variables for Colab
import os
os.environ['PYTHONPATH'] = '/content/coding-framework/src'
os.environ['COLAB_MODE'] = 'true'
os.environ['HF_HOME'] = '/content/model_cache'
os.environ['TRANSFORMERS_CACHE'] = '/content/model_cache'

# Optional: Set HuggingFace token if you want to access gated models
# os.environ['HUGGINGFACE_HUB_TOKEN'] = 'your_token_here'

print("✅ Environment variables configured")

## 🧠 Model Selection for Colab

Choose the appropriate model based on your Colab resources:

In [ ]:
# Model options for different Colab tiers
MODELS = {
    "lightweight": {
        "name": "microsoft/DialoGPT-small",
        "description": "Lightweight model, works on free Colab",
        "memory": "~1GB"
    },
    "medium": {
        "name": "microsoft/CodeGPT-small-py", 
        "description": "Code-focused model, good balance",
        "memory": "~2GB"
    },
    "advanced": {
        "name": "Salesforce/codegen-350M-multi",
        "description": "Multi-language code generation",
        "memory": "~3GB"
    },
    "powerful": {
        "name": "bigcode/starcoder2-3b",
        "description": "High-quality code generation (Colab Pro)",
        "memory": "~6GB"
    }
}

# Display model options
print("Available models for Colab:")
for key, model in MODELS.items():
    print(f"\n{key.upper()}:")
    print(f"  Model: {model['name']}")
    print(f"  Description: {model['description']}")
    print(f"  Memory: {model['memory']}")

# Select model based on GPU availability
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    if gpu_memory > 14:  # Colab Pro/Pro+
        selected_model = MODELS["powerful"]
    elif gpu_memory > 10:  # Standard GPU
        selected_model = MODELS["advanced"]
    else:
        selected_model = MODELS["medium"]
else:
    selected_model = MODELS["lightweight"]

print(f"\n🎯 Selected model: {selected_model['name']}")
print(f"📝 Description: {selected_model['description']}")

## ⚙️ Framework Configuration

Set up the framework with Colab-optimized configuration:

In [ ]:
# Create Colab-specific configuration
from src.coding_framework.utils.config import Config, LLMConfig
import yaml

# Load base Colab config
with open('config/colab_config.yaml', 'r') as f:
    config_data = yaml.safe_load(f)

# Update with selected model
config_data['llm']['model'] = selected_model['name']

# Save updated config
with open('/content/colab_runtime_config.yaml', 'w') as f:
    yaml.dump(config_data, f, default_flow_style=False)

# Load configuration
config = Config(**config_data)

print("✅ Configuration created for Colab")
print(f"🧠 Using model: {config.llm.model}")
print(f"🔧 Provider: {config.llm.provider}")
print(f"🎛️ Max tokens: {config.llm.max_tokens}")

In [ ]:
# Initialize the framework
import asyncio
from src.coding_framework.orchestration import CodingSupervisor
from src.coding_framework.utils import setup_logging

# Setup logging for Colab
setup_logging(level="INFO", verbose=True, format_type="text")

# Initialize supervisor
supervisor = CodingSupervisor(config)

print("🎯 Initializing framework components...")
print("⚠️  This may take a few minutes to download and load the model...")

# Initialize in async context
async def init_framework():
    await supervisor.initialize()
    return await supervisor.health_check()

# Run initialization
health_status = await init_framework()

print("\n✅ Framework initialized successfully!")
print(f"🏥 System health: {health_status['system']['status']}")

## 🎯 Demo: Multi-Agent Code Generation

Let's demonstrate the framework with a complete coding workflow:

In [ ]:
# Demo 1: Simple problem solving
from rich.console import Console
from rich.panel import Panel
from rich.syntax import Syntax

console = Console()

async def demo_simple_problem():
    problem = "Write a Python function to calculate the factorial of a number"
    
    console.print(Panel.fit(
        f"[bold blue]Problem:[/bold blue] {problem}",
        border_style="blue"
    ))
    
    # Solve the problem
    result = await supervisor.solve_problem(
        problem,
        context={
            "language": "python",
            "style": "clean",
            "include_tests": False
        }
    )
    
    if result["success"]:
        # Display generated code
        console.print("\n[bold green]✅ Generated Code:[/bold green]")
        syntax = Syntax(result["code"], "python", theme="github-dark")
        console.print(syntax)
        
        # Display review
        if result.get("review"):
            console.print("\n[bold yellow]📝 Code Review:[/bold yellow]")
            console.print(result["review"])
        
        # Display execution results
        if result.get("execution"):
            console.print("\n[bold cyan]⚡ Execution Results:[/bold cyan]")
            console.print(result["execution"])
        
        console.print(f"\n[dim]⏱️  Total time: {result.get('execution_time', 0):.2f}s[/dim]")
    else:
        console.print(f"[bold red]❌ Error:[/bold red] {result.get('error')}")
    
    return result

# Run the demo
simple_result = await demo_simple_problem()

In [ ]:
# Demo 2: More complex problem with multiple iterations
async def demo_complex_problem():
    problem = """Create a Python class that implements a simple calculator with the following features:
    1. Basic operations (add, subtract, multiply, divide)
    2. Memory functions (store, recall, clear)
    3. Error handling for division by zero
    4. Method chaining support
    """
    
    console.print(Panel.fit(
        f"[bold blue]Complex Problem:[/bold blue]\n{problem}",
        border_style="blue"
    ))
    
    # Solve with enhanced context
    result = await supervisor.solve_problem(
        problem,
        context={
            "language": "python",
            "style": "clean",
            "include_tests": False,
            "max_iterations": 3
        }
    )
    
    if result["success"]:
        console.print("\n[bold green]✅ Generated Calculator Class:[/bold green]")
        syntax = Syntax(result["code"], "python", theme="github-dark")
        console.print(syntax)
        
        console.print(f"\n[bold magenta]📊 Metrics:[/bold magenta]")
        console.print(f"• Iterations: {result.get('iterations', 'N/A')}")
        console.print(f"• Review Score: {result.get('review_score', 'N/A')}/100")
        console.print(f"• Execution Success: {result.get('execution_success', 'N/A')}")
    else:
        console.print(f"[bold red]❌ Error:[/bold red] {result.get('error')}")
    
    return result

# Run the complex demo
complex_result = await demo_complex_problem()

In [ ]:
# Demo 3: Individual agent testing
async def test_individual_agents():
    console.print(Panel.fit(
        "[bold purple]Testing Individual Agents[/bold purple]",
        border_style="purple"
    ))
    
    # Test Code Generator
    console.print("\n[bold]1. Testing Code Generator Agent:[/bold]")
    gen_result = await supervisor.generate_code(
        "Write a function to check if a number is prime",
        context={"language": "python"}
    )
    
    if gen_result["success"]:
        console.print("[green]✅ Generator working[/green]")
        print(f"Generated: {len(gen_result['content'])} characters")
    else:
        console.print("[red]❌ Generator failed[/red]")
    
    # Test Code Reviewer
    console.print("\n[bold]2. Testing Code Reviewer Agent:[/bold]")
    sample_code = '''def factorial(n):
    if n == 0:
        return 1
    return n * factorial(n-1)'''
    
    review_result = await supervisor.review_code(sample_code)
    
    if review_result["success"]:
        console.print("[green]✅ Reviewer working[/green]")
        score = review_result.get("metadata", {}).get("overall_score", "N/A")
        print(f"Review score: {score}/100")
    else:
        console.print("[red]❌ Reviewer failed[/red]")
    
    # Test Code Executor
    console.print("\n[bold]3. Testing Code Executor Agent:[/bold]")
    simple_code = "print('Hello from the executor!')"
    
    exec_result = await supervisor.execute_code(
        simple_code,
        context={"language": "python"}
    )
    
    if exec_result["success"]:
        console.print("[green]✅ Executor working[/green]")
    else:
        console.print("[red]❌ Executor failed[/red]")
        console.print(f"Error: {exec_result.get('error')}")

# Test individual agents
await test_individual_agents()

## 📊 Performance Monitoring

Monitor system performance during execution:

In [ ]:
# Performance monitoring
import psutil
import GPUtil
from IPython.display import clear_output
import time
import matplotlib.pyplot as plt

def get_system_stats():
    """Get current system statistics."""
    stats = {
        'cpu_percent': psutil.cpu_percent(interval=1),
        'memory_percent': psutil.virtual_memory().percent,
        'disk_percent': psutil.disk_usage('/').percent
    }
    
    # GPU stats if available
    try:
        gpus = GPUtil.getGPUs()
        if gpus:
            stats['gpu_percent'] = gpus[0].load * 100
            stats['gpu_memory_percent'] = (gpus[0].memoryUsed / gpus[0].memoryTotal) * 100
    except:
        stats['gpu_percent'] = 0
        stats['gpu_memory_percent'] = 0
    
    return stats

# Get current performance metrics
stats = get_system_stats()
performance_metrics = supervisor.get_performance_metrics()

console.print(Panel.fit(
    f"""[bold cyan]System Performance[/bold cyan]

[bold]Resource Usage:[/bold]
🔥 CPU: {stats['cpu_percent']:.1f}%
🧠 RAM: {stats['memory_percent']:.1f}%
💾 Disk: {stats['disk_percent']:.1f}%
🎮 GPU: {stats['gpu_percent']:.1f}%
📊 GPU Memory: {stats['gpu_memory_percent']:.1f}%

[bold]Framework Metrics:[/bold]
✅ Problems Solved: {performance_metrics.get('total_problems_solved', 0)}
🎯 Success Rate: {(performance_metrics.get('successful_solutions', 0) / max(1, performance_metrics.get('total_problems_solved', 1)) * 100):.1f}%
⏱️  Avg Response Time: {performance_metrics.get('avg_execution_time', 0):.2f}s
📝 Avg Review Score: {performance_metrics.get('avg_review_score', 0):.1f}/100
""",
    border_style="cyan"
))

## 🧪 Running Tests

Let's run some basic tests to ensure everything works correctly:

In [ ]:
# Run basic framework tests
import subprocess
import sys

console.print(Panel.fit(
    "[bold green]Running Framework Tests[/bold green]",
    border_style="green"
))

# Test configuration loading
print("1. Testing configuration loading...")
try:
    from src.coding_framework.utils.config import load_config
    test_config = load_config('/content/colab_runtime_config.yaml')
    assert test_config.llm.provider == "huggingface"
    print("   ✅ Configuration test passed")
except Exception as e:
    print(f"   ❌ Configuration test failed: {e}")

# Test LLM interface
print("\n2. Testing LLM interface...")
try:
    from src.coding_framework.utils.llm_interface import LLMInterface
    llm_interface = LLMInterface(test_config.llm)
    health = await llm_interface.health_check()
    print(f"   ✅ LLM interface test passed: {health['status']}")
except Exception as e:
    print(f"   ❌ LLM interface test failed: {e}")

# Test agent initialization
print("\n3. Testing agent initialization...")
try:
    health_status = await supervisor.health_check()
    healthy_agents = sum(1 for agent, status in health_status.items() 
                        if isinstance(status, dict) and status.get('status') == 'healthy')
    print(f"   ✅ Agent initialization test passed: {healthy_agents} healthy agents")
except Exception as e:
    print(f"   ❌ Agent initialization test failed: {e}")

# Test basic workflow
print("\n4. Testing basic workflow...")
try:
    simple_test = await supervisor.solve_problem(
        "Write a function that returns 'Hello World'",
        context={"language": "python", "max_iterations": 1}
    )
    if simple_test["success"]:
        print("   ✅ Basic workflow test passed")
    else:
        print(f"   ⚠️  Basic workflow test completed with issues: {simple_test.get('error')}")
except Exception as e:
    print(f"   ❌ Basic workflow test failed: {e}")

print("\n🎉 Test suite completed!")

In [ ]:
# Run selected unit tests (modified for Colab)
print("Running unit tests (Colab-compatible subset)...")

# Test imports
try:
    from src.coding_framework import (
        CodeGeneratorAgent,
        CodeReviewerAgent,
        CodeExecutorAgent,
        CodingSupervisor
    )
    print("✅ All core imports successful")
except ImportError as e:
    print(f"❌ Import error: {e}")

# Test agent type properties
try:
    generator = supervisor.agents["generator"]
    reviewer = supervisor.agents["reviewer"]
    executor = supervisor.agents["executor"]
    
    assert generator.agent_type == "code_generator"
    assert reviewer.agent_type == "code_reviewer"
    assert executor.agent_type == "code_executor"
    
    print("✅ Agent type tests passed")
except Exception as e:
    print(f"❌ Agent type test failed: {e}")

# Test configuration validation
try:
    from src.coding_framework.utils.config import validate_config
    issues = validate_config(test_config)
    if not issues:
        print("✅ Configuration validation passed")
    else:
        print(f"⚠️  Configuration has issues: {issues}")
except Exception as e:
    print(f"❌ Configuration validation failed: {e}")

print("\n🧪 Unit tests completed!")

## 💾 Save Results and Models

Save your work to Google Drive:

In [ ]:
# Save results and configuration to Drive
import json
from datetime import datetime

# Prepare results summary
results_summary = {
    "timestamp": datetime.now().isoformat(),
    "model_used": selected_model["name"],
    "system_info": {
        "gpu_available": torch.cuda.is_available(),
        "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None",
        "colab_type": "GPU" if torch.cuda.is_available() else "CPU"
    },
    "performance_metrics": supervisor.get_performance_metrics(),
    "demo_results": {
        "simple_problem": {
            "success": simple_result.get("success", False),
            "execution_time": simple_result.get("execution_time", 0)
        },
        "complex_problem": {
            "success": complex_result.get("success", False),
            "execution_time": complex_result.get("execution_time", 0),
            "iterations": complex_result.get("iterations", 0)
        }
    }
}

# Save to Drive
output_dir = '/content/drive/MyDrive/coding_framework/colab_session'
os.makedirs(output_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save results
with open(f'{output_dir}/results_{timestamp}.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

# Save configuration
with open(f'{output_dir}/config_{timestamp}.yaml', 'w') as f:
    yaml.dump(config.dict(), f, default_flow_style=False)

# Save demo code examples
if simple_result.get("success"):
    with open(f'{output_dir}/generated_code_{timestamp}.py', 'w') as f:
        f.write(f"# Simple Problem Solution\n")
        f.write(f"# Generated at: {datetime.now()}\n")
        f.write(f"# Model: {selected_model['name']}\n\n")
        f.write(simple_result["code"])
        
        if complex_result.get("success"):
            f.write(f"\n\n# Complex Problem Solution\n")
            f.write(complex_result["code"])

console.print(Panel.fit(
    f"""[bold green]✅ Session Saved Successfully![/bold green]

[bold]Saved Files:[/bold]
📊 Results: results_{timestamp}.json
⚙️  Config: config_{timestamp}.yaml
🐍 Code: generated_code_{timestamp}.py

[bold]Location:[/bold] {output_dir}

[dim]You can access these files from your Google Drive.[/dim]
""",
    border_style="green"
))

print(f"\n🎯 Total problems solved: {supervisor.get_performance_metrics().get('total_problems_solved', 0)}")
print(f"⏱️  Average response time: {supervisor.get_performance_metrics().get('avg_execution_time', 0):.2f}s")
print(f"📝 Average review score: {supervisor.get_performance_metrics().get('avg_review_score', 0):.1f}/100")

## 🚀 Next Steps

Now that you have the framework running in Colab, here are some things you can try:

### 🎯 **Experiment with Different Problems**
```python
# Try more complex problems
problems = [
    "Implement a binary search tree with insert, delete, and search operations",
    "Create a web scraper that respects robots.txt",
    "Write a function to find the longest common subsequence",
    "Implement a simple neural network from scratch"
]

for problem in problems:
    result = await supervisor.solve_problem(problem, context={"language": "python"})
    # Process results...
```

### 🔧 **Customize Configuration**
- Modify `config/colab_config.yaml` for different settings
- Try different HuggingFace models
- Adjust workflow parameters

### 🧠 **Try Different Models**
- Code-specific models: `bigcode/starcoder2-3b`
- Multi-language models: `Salesforce/codegen-350M-multi`
- Instruction-tuned models: `microsoft/DialoGPT-medium`

### 📊 **Monitor Performance**
- Use the performance monitoring cells above
- Track memory usage with different model sizes
- Compare generation quality across models

### 🚀 **Scale Up (Colab Pro)**
- Use larger models with more GPU memory
- Enable VERL training (coming soon)
- Process multiple problems in parallel

---

## 📚 Resources

- **Framework Documentation**: [GitHub Repository](https://github.com/multiminddev/coding-framework)
- **HuggingFace Models**: [Hub](https://huggingface.co/models?pipeline_tag=text-generation)
- **LangGraph Documentation**: [LangChain Docs](https://langchain-ai.github.io/langgraph/)
- **VERL Framework**: [VERL GitHub](https://github.com/volcengine/verl)

## 🐛 Troubleshooting

If you encounter issues:

1. **Memory Issues**: Use smaller models or restart runtime
2. **Model Loading**: Check internet connection and HuggingFace status
3. **CUDA Errors**: Restart runtime and ensure GPU is enabled
4. **Import Errors**: Reinstall dependencies with `!pip install -r requirements-colab.txt`

---

**🎉 Congratulations!** You've successfully set up and tested the VERL + LangGraph Multi-Agent Coding Framework in Google Colab with HuggingFace models! 🚀